In [4]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
import numpy as np
import os

Using TensorFlow backend.


In [5]:
%run data_loader.ipynb

In [6]:
class CycleGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'horse2zebra'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=1, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])
                    
                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i, self.data_loader.n_batches,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
            
            self.combined.save('cyclegan_models/v1_horse2zebra/epoch_' + str(epoch) + '_model.h5')
                
    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

In [ ]:
if __name__ == '__main__':
    gan = CycleGAN()
    gan.train(epochs=200, batch_size=4, sample_interval=200)

/home/amrit/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/amrit/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/amrit/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/200] [Batch 0/266] [D loss: 28.902859, acc:  10%] [G loss: 54.874325, adv: 19.892166, recon: 0.667843, id: 1.059934] time: 0:00:54.126151 
[Epoch 0/200] [Batch 1/266] [D loss: 9.926638, acc:  15%] [G loss: 68.096985, adv: 26.460175, recon: 0.670909, id: 0.911474] time: 0:00:55.201686 
[Epoch 0/200] [Batch 2/266] [D loss: 11.332312, acc:  11%] [G loss: 41.906540, adv: 13.890949, recon: 0.627404, id: 0.833495] time: 0:00:55.842391 
[Epoch 0/200] [Batch 3/266] [D loss: 6.562084, acc:  18%] [G loss: 19.063219, adv: 2.634443, recon: 0.605820, id: 0.857869] time: 0:00:56.430360 
[Epoch 0/200] [Batch 4/266] [D loss: 3.504441, acc:  19%] [G loss: 14.399940, adv: 1.114183, recon: 0.511620, id: 1.046693] time: 0:00:57.052834 
[Epoch 0/200] [Batch 5/266] [D loss: 1.221640, acc:  36%] [G loss: 13.581953, adv: 0.924550, recon: 0.495146, id: 0.952461] time: 0:00:57.724454 
[Epoch 0/200] [Batch 6/266] [D loss: 0.483917, acc:  52%] [G loss: 13.349558, adv: 0.963545, recon: 0.476792, id: 0.982

[Epoch 0/200] [Batch 56/266] [D loss: 0.519890, acc:  24%] [G loss: 8.090616, adv: 0.570924, recon: 0.252512, id: 1.062010] time: 0:01:27.908569 
[Epoch 0/200] [Batch 57/266] [D loss: 0.411220, acc:  42%] [G loss: 9.605376, adv: 0.895550, recon: 0.304995, id: 0.987738] time: 0:01:28.493266 
[Epoch 0/200] [Batch 58/266] [D loss: 0.397164, acc:  55%] [G loss: 9.717443, adv: 0.611568, recon: 0.333658, id: 0.879348] time: 0:01:29.028926 
[Epoch 0/200] [Batch 59/266] [D loss: 0.513797, acc:  27%] [G loss: 8.612302, adv: 0.579173, recon: 0.289718, id: 0.770504] time: 0:01:29.683829 
[Epoch 0/200] [Batch 60/266] [D loss: 0.555678, acc:  37%] [G loss: 8.305759, adv: 1.254650, recon: 0.203373, id: 0.830766] time: 0:01:30.199675 
[Epoch 0/200] [Batch 61/266] [D loss: 0.643188, acc:  50%] [G loss: 8.028234, adv: 0.627510, recon: 0.262854, id: 0.710464] time: 0:01:30.759984 
[Epoch 0/200] [Batch 62/266] [D loss: 0.616676, acc:  29%] [G loss: 8.396306, adv: 0.540702, recon: 0.291598, id: 0.616187] 

[Epoch 0/200] [Batch 113/266] [D loss: 0.253790, acc:  65%] [G loss: 8.081391, adv: 0.747204, recon: 0.264461, id: 0.666529] time: 0:02:01.931049 
[Epoch 0/200] [Batch 114/266] [D loss: 0.384032, acc:  40%] [G loss: 7.390393, adv: 0.574991, recon: 0.240305, id: 0.665329] time: 0:02:02.600499 
[Epoch 0/200] [Batch 115/266] [D loss: 0.327503, acc:  43%] [G loss: 6.675506, adv: 0.507846, recon: 0.197729, id: 0.813417] time: 0:02:03.253977 
[Epoch 0/200] [Batch 116/266] [D loss: 0.379796, acc:  43%] [G loss: 7.225586, adv: 0.560857, recon: 0.217600, id: 0.906865] time: 0:02:03.853818 
[Epoch 0/200] [Batch 117/266] [D loss: 0.287697, acc:  52%] [G loss: 8.072680, adv: 0.695947, recon: 0.236946, id: 0.896232] time: 0:02:04.408603 
[Epoch 0/200] [Batch 118/266] [D loss: 0.361924, acc:  49%] [G loss: 8.200281, adv: 0.668147, recon: 0.275819, id: 0.707033] time: 0:02:04.981412 
[Epoch 0/200] [Batch 119/266] [D loss: 0.265554, acc:  63%] [G loss: 7.209681, adv: 0.672283, recon: 0.218735, id: 0.7

[Epoch 0/200] [Batch 169/266] [D loss: 0.233580, acc:  59%] [G loss: 6.799505, adv: 0.584661, recon: 0.212660, id: 0.716462] time: 0:02:34.873464 
[Epoch 0/200] [Batch 170/266] [D loss: 0.237954, acc:  57%] [G loss: 7.201649, adv: 0.574346, recon: 0.218014, id: 0.928896] time: 0:02:35.505237 
[Epoch 0/200] [Batch 171/266] [D loss: 0.391844, acc:  31%] [G loss: 6.772981, adv: 0.456665, recon: 0.208826, id: 0.722113] time: 0:02:36.075531 
[Epoch 0/200] [Batch 172/266] [D loss: 0.353324, acc:  43%] [G loss: 6.092183, adv: 0.405900, recon: 0.183796, id: 0.823814] time: 0:02:36.625696 
[Epoch 0/200] [Batch 173/266] [D loss: 0.324239, acc:  39%] [G loss: 6.474831, adv: 0.449843, recon: 0.201417, id: 0.778055] time: 0:02:37.201998 
[Epoch 0/200] [Batch 174/266] [D loss: 0.343320, acc:  42%] [G loss: 6.773449, adv: 0.531767, recon: 0.215923, id: 0.729445] time: 0:02:37.720279 
[Epoch 0/200] [Batch 175/266] [D loss: 0.282432, acc:  49%] [G loss: 6.498951, adv: 0.619570, recon: 0.194820, id: 0.6

...

[Epoch 4/200] [Batch 67/266] [D loss: 0.156254, acc:  73%] [G loss: 6.466966, adv: 0.830551, recon: 0.156546, id: 0.825696] time: 0:10:13.040154 
[Epoch 4/200] [Batch 68/266] [D loss: 0.272604, acc:  60%] [G loss: 6.087988, adv: 0.817138, recon: 0.145347, id: 0.770519] time: 0:10:13.486879 
[Epoch 4/200] [Batch 69/266] [D loss: 0.248847, acc:  64%] [G loss: 5.695304, adv: 0.689683, recon: 0.148478, id: 0.651078] time: 0:10:13.942469 
[Epoch 4/200] [Batch 70/266] [D loss: 0.208303, acc:  73%] [G loss: 5.346338, adv: 0.742312, recon: 0.122934, id: 0.685716] time: 0:10:14.388207 
[Epoch 4/200] [Batch 71/266] [D loss: 0.275812, acc:  59%] [G loss: 5.145421, adv: 0.559521, recon: 0.131692, id: 0.754686] time: 0:10:14.833401 
[Epoch 4/200] [Batch 72/266] [D loss: 0.172406, acc:  73%] [G loss: 5.300768, adv: 0.615753, recon: 0.135529, id: 0.724516] time: 0:10:15.291705 
[Epoch 4/200] [Batch 73/266] [D loss: 0.343433, acc:  57%] [G loss: 5.325161, adv: 0.524031, recon: 0.151493, id: 0.609802] 

[Epoch 4/200] [Batch 123/266] [D loss: 0.210443, acc:  69%] [G loss: 4.955574, adv: 0.772820, recon: 0.108769, id: 0.513921] time: 0:10:38.535737 
[Epoch 4/200] [Batch 124/266] [D loss: 0.128643, acc:  82%] [G loss: 5.274697, adv: 0.788186, recon: 0.123462, id: 0.557573] time: 0:10:38.990586 
[Epoch 4/200] [Batch 125/266] [D loss: 0.138422, acc:  83%] [G loss: 5.713815, adv: 0.694280, recon: 0.142507, id: 0.658209] time: 0:10:39.440966 
[Epoch 4/200] [Batch 126/266] [D loss: 0.145839, acc:  77%] [G loss: 6.141931, adv: 0.774301, recon: 0.147407, id: 0.722841] time: 0:10:39.888462 
[Epoch 4/200] [Batch 127/266] [D loss: 0.199878, acc:  68%] [G loss: 5.168349, adv: 0.669888, recon: 0.122799, id: 0.636409] time: 0:10:40.337649 
[Epoch 4/200] [Batch 128/266] [D loss: 0.160471, acc:  77%] [G loss: 5.062796, adv: 0.658892, recon: 0.113303, id: 0.721370] time: 0:10:40.789331 
[Epoch 4/200] [Batch 129/266] [D loss: 0.163340, acc:  75%] [G loss: 5.017574, adv: 0.725753, recon: 0.115573, id: 0.5

[Epoch 4/200] [Batch 179/266] [D loss: 0.258391, acc:  61%] [G loss: 5.703497, adv: 0.734005, recon: 0.137011, id: 0.646322] time: 0:11:03.944488 
[Epoch 4/200] [Batch 180/266] [D loss: 0.242625, acc:  60%] [G loss: 5.423168, adv: 0.558554, recon: 0.140503, id: 0.615800] time: 0:11:04.407988 
[Epoch 4/200] [Batch 181/266] [D loss: 0.204780, acc:  75%] [G loss: 5.327824, adv: 0.553311, recon: 0.144365, id: 0.613528] time: 0:11:04.873314 
[Epoch 4/200] [Batch 182/266] [D loss: 0.287047, acc:  53%] [G loss: 4.925427, adv: 0.512970, recon: 0.121581, id: 0.678318] time: 0:11:05.326358 
[Epoch 4/200] [Batch 183/266] [D loss: 0.230365, acc:  65%] [G loss: 5.183632, adv: 0.596181, recon: 0.126243, id: 0.603780] time: 0:11:05.782853 
[Epoch 4/200] [Batch 184/266] [D loss: 0.109868, acc:  91%] [G loss: 5.227447, adv: 0.662207, recon: 0.137512, id: 0.583201] time: 0:11:06.242076 
[Epoch 4/200] [Batch 185/266] [D loss: 0.180405, acc:  74%] [G loss: 4.847363, adv: 0.607315, recon: 0.118349, id: 0.5

[Epoch 4/200] [Batch 235/266] [D loss: 0.221160, acc:  66%] [G loss: 5.211889, adv: 0.635311, recon: 0.132287, id: 0.567584] time: 0:11:29.649176 
[Epoch 4/200] [Batch 236/266] [D loss: 0.117160, acc:  84%] [G loss: 5.381466, adv: 0.761226, recon: 0.130752, id: 0.588701] time: 0:11:30.101777 
[Epoch 4/200] [Batch 237/266] [D loss: 0.124608, acc:  84%] [G loss: 6.059268, adv: 0.769306, recon: 0.152672, id: 0.636458] time: 0:11:30.554922 
[Epoch 4/200] [Batch 238/266] [D loss: 0.120591, acc:  86%] [G loss: 6.196877, adv: 0.864194, recon: 0.148734, id: 0.560165] time: 0:11:31.007148 
[Epoch 4/200] [Batch 239/266] [D loss: 0.111140, acc:  86%] [G loss: 5.515707, adv: 0.775239, recon: 0.129124, id: 0.635131] time: 0:11:31.463456 
[Epoch 4/200] [Batch 240/266] [D loss: 0.222771, acc:  62%] [G loss: 5.160761, adv: 0.570601, recon: 0.126450, id: 0.748254] time: 0:11:31.915053 
[Epoch 4/200] [Batch 241/266] [D loss: 0.166690, acc:  76%] [G loss: 4.971771, adv: 0.687889, recon: 0.111798, id: 0.5

[Epoch 5/200] [Batch 26/266] [D loss: 0.221628, acc:  64%] [G loss: 4.902951, adv: 0.587043, recon: 0.121860, id: 0.611997] time: 0:11:55.881796 
[Epoch 5/200] [Batch 27/266] [D loss: 0.109049, acc:  88%] [G loss: 5.210799, adv: 0.757497, recon: 0.122203, id: 0.559949] time: 0:11:56.335470 
[Epoch 5/200] [Batch 28/266] [D loss: 0.114249, acc:  84%] [G loss: 5.917319, adv: 0.777395, recon: 0.142350, id: 0.735386] time: 0:11:56.788531 
[Epoch 5/200] [Batch 29/266] [D loss: 0.163227, acc:  75%] [G loss: 5.150533, adv: 0.671221, recon: 0.119295, id: 0.537726] time: 0:11:57.242331 
[Epoch 5/200] [Batch 30/266] [D loss: 0.137247, acc:  81%] [G loss: 4.758477, adv: 0.707804, recon: 0.101675, id: 0.584977] time: 0:11:57.695252 
[Epoch 5/200] [Batch 31/266] [D loss: 0.165120, acc:  78%] [G loss: 5.502893, adv: 0.823614, recon: 0.123851, id: 0.565228] time: 0:11:58.149888 
[Epoch 5/200] [Batch 32/266] [D loss: 0.118056, acc:  84%] [G loss: 5.594052, adv: 0.877763, recon: 0.120842, id: 0.694900] 